In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Table of Contents

1. Basic autograd example 1               (Line 21 to 36)
2. Basic autograd example 2               (Line 39 to 77)
3. Loading data from numpy                (Line 80 to 83)
4. Implementing the input pipline         (Line 86 to 113)
5. Input pipline for custom dataset       (Line 116 to 138)
6. Using pretrained model                 (Line 141 to 155)
7. Save and load model                    (Line 158 to 165) 


In [10]:
x = Variable(torch.Tensor([1]), requires_grad = True)
w = Variable(torch.Tensor([2]), requires_grad = True)
b = Variable(torch.Tensor([3]), requires_grad = True)

In [11]:
y = w * x + b

In [12]:
y.backward()

In [15]:
# Print out the gradients.
print(x.grad)    # x.grad = dy/dx = 2 
print(w.grad)    # w.grad = dy/dw = 1 
print(b.grad)    # b.grad = dy/db = 1 

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



# Create a Liner Layer

`torch.nn.Linear(feature_in, feature_out, bias = True)` applies linear transformation `out = weight * x + bias` to incoming data `x`.

Parameters:
* weight
* bias

In [18]:
x = Variable(torch.randn(5,3))
y = Variable(torch.randn(5,2))

linear = nn.Linear(3,2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
-0.3535 -0.0594  0.5687
-0.0945 -0.5151  0.5412
[torch.FloatTensor of size 2x3]

b:  Parameter containing:
 0.1333
 0.2282
[torch.FloatTensor of size 2]



Define MSE Loss function and SGD optimizer

In [19]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.01)

In [21]:
pred = linear(x)
pred

Variable containing:
 0.6071  0.3219
 0.5879  0.5524
-1.3501 -0.8461
-0.9466 -0.8917
 0.1905 -0.1409
[torch.FloatTensor of size 5x2]

In [26]:
loss = criterion(pred, y)
loss.data


 0.9466
[torch.FloatTensor of size 1]

In [27]:
loss.backward()

In [28]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  Variable containing:
-0.1622  0.6534  0.2609
 0.1667  0.2528 -0.0888
[torch.FloatTensor of size 2x3]

dL/db:  Variable containing:
 0.8561
 0.3705
[torch.FloatTensor of size 2]



In [29]:
optimizer.step()

In [30]:
# Print out the loss after optimization.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.data[0])

loss after 1 step optimization:  0.931778609752655


In [31]:
loss.backward()
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  Variable containing:
-0.3253  1.3008  0.5194
 0.3302  0.5030 -0.1744
[torch.FloatTensor of size 2x3]

dL/db:  Variable containing:
 1.7020
 0.7354
[torch.FloatTensor of size 2]



In [32]:
optimizer.step()
# Print out the loss after optimization.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 2 step optimization: ', loss.data[0])

loss after 2 step optimization:  0.9027829170227051


In [34]:
a =  np.array([[1,2],[3,4]])
b = torch.from_numpy(a)
c = b.numpy()
print(a)
print(b)
print(c)

[[1 2]
 [3 4]]

 1  2
 3  4
[torch.LongTensor of size 2x2]

[[1 2]
 [3 4]]


# Implementing Input Pipeline.

In [36]:
training_data = dsets.CIFAR10('../data/', train = True, transform = transforms.ToTensor(),download = True)
image, label = training_data[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


Datasets can be multithreaded using `torch.utils.data.DataLoader`

In [39]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size = 100, shuffle = True, num_workers = 2)
data_iter = iter(train_loader)
images, labels = data_iter.next()

print(images.size())

torch.Size([100, 3, 32, 32])


# Defining custom Dataset

In [40]:
#===================== Input pipline for custom dataset =====================#
# You should build custom dataset as below.
class CustomDataset(data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file path or list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# Then, you can just use prebuilt torch's data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)
